In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.metrics import root_mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experimental")

<Experiment: artifact_location='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns/2', creation_time=1748046038801, experiment_id='2', last_update_time=1748046038801, lifecycle_stage='active', name='nyc-taxi-experimental', tags={}>

In [3]:
!mlflow --version

mlflow, version 2.22.0


In [5]:
df = pd.read_parquet("../data/green/green_tripdata_2023-01.parquet")

In [6]:
df_val = pd.read_parquet("../data/green/green_tripdata_2023-02.parquet")

In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60
df = df[df.duration >= 1 & (df.duration < 60)]

train_dicts = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

/tmp/ipykernel_20334/3886218009.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
/tmp/ipykernel_20334/3886218009.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.duration = df.duration.dt.total_seconds() / 60


In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

rmse = root_mean_squared_error(y_train, y_pred)

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr, f_out))

FileNotFoundError: [Errno 2] No such file or directory: 'models/lin_reg.bin'

In [11]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "pastor")
    mlflow.log_param("train-data-path", "../data/green/green_tripdata_2023-01.parquet")
    

    
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_train)
    
    rmse = root_mean_squared_error(y_train, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle/")

MlflowException: Invalid artifact path: 'models_pickle/'. Names may be treated as files in certain cases, and must not resolve to other names when treated as such. This name would resolve to 'models_pickle'

In [17]:
def preprocess(df: pd.DataFrame, dv: DictVectorizer, fit_dv: bool = False):
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[df.duration >= 1 & (df.duration < 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']
    target = 'duration'
    
    df[categorical] = df[categorical].astype(str)

    
    y = df[target].values

    dicts = df[categorical + numerical].to_dict(orient='records')
    if fit_dv:
        X = dv.fit_transform(dicts)
    else:
        X = dv.transform(dicts)
    return X, y, dv


In [18]:
dv = DictVectorizer()
X_train, y_train, dv = preprocess(df, dv, fit_dv=True)

In [19]:
X_val, y_val, _ = preprocess(df_val, dv, fit_dv=False)

/tmp/ipykernel_20334/3974920714.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [20]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)

In [22]:
valid = xgb.DMatrix(X_val, label=y_val)

In [40]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}
            

In [44]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),  # exp(-3), exp(0) [0.005 - 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),  # exp(-5), exp(-1) 0.00001 - 0.1
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),  # exp(-6), exp(-1) 0.000001 - 0.1
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objevtive': 'reg_linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials())
    

  0%|                                                                                                                 | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [18:24:08] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "objevtive" } are not used.

  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:70.04574                                                                                                                                 
[1]	validation-rmse:70.13097                                                                                                                                 
[2]	validation-rmse:70.33594                                                                                                                                 
[3]	validation-rmse:70.58692                                                                                                                                 
[4]	validation-rmse:70.90579                                                                                                                                 
[5]	validation-rmse:71.31401                                                                                                                                 
[6]	validation-rmse:71.72145                        

/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [18:24:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "objevtive" } are not used.

  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:69.89906                                                                                                                                 
[3]	validation-rmse:69.88788                                                                                                                                 
[4]	validation-rmse:69.88548                                                                                                                                 
[5]	validation-rmse:69.90460                                                                                                                                 
[6]	validation-rmse:69.92343                                                                                                                                 
[7]	validation-rmse:69.95123                                                                                                                                 
[8]	validation-rmse:69.98697                        

...

[0]	validation-rmse:71.32241                                                                                                                
[1]	validation-rmse:73.25065                                                                                                                
[2]	validation-rmse:74.95032                                                                                                                
[3]	validation-rmse:76.29575                                                                                                                
[4]	validation-rmse:77.14286                                                                                                                
[5]	validation-rmse:77.70828                                                                                                                
[6]	validation-rmse:78.13244                                                                                                                
[7]	validatio

/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [18:27:49] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "objevtive" } are not used.

  self.starting_round = model.num_boosted_rounds()



[3]	validation-rmse:69.98641                                                                                                                
[4]	validation-rmse:70.01267                                                                                                                
[5]	validation-rmse:70.05697                                                                                                                
[6]	validation-rmse:70.10391                                                                                                                
[7]	validation-rmse:70.16748                                                                                                                
[8]	validation-rmse:70.23316                                                                                                                
[9]	validation-rmse:70.29511                                                                                                                
[10]	validati

/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [18:27:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "objevtive" } are not used.

  self.starting_round = model.num_boosted_rounds()



[8]	validation-rmse:70.09183                                                                                                                
[9]	validation-rmse:70.11602                                                                                                                
[10]	validation-rmse:70.15310                                                                                                               
[11]	validation-rmse:70.19078                                                                                                               
[12]	validation-rmse:70.22945                                                                                                               
[13]	validation-rmse:70.24185                                                                                                               
[14]	validation-rmse:70.25423                                                                                                               
[15]	validati

In [31]:
with mlflow.start_run():  
    parameters = {
        'learning_rate': 0.05380677154145605,
        'max_depth': 4,
        'min_child_weight': 3.754106686202785,
        'objective': 'reg:linear',  # Fixed typo from 'objevtive'
        'reg_alpha': 0.07023067397868384,
        'reg_lambda': 0.0024864712355091413,
        'seed': 42
    }
    
    #mlflow.xgboost.autolog()
    mlflow.log_params(parameters)
    booster = xgb.train(
        params = parameters,
        dtrain = train,
        num_boost_round = 100,
        evals = [(valid, "validation")],
        early_stopping_rounds = 50
    )
    
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    with open("mlruns/models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("mlruns/models/preprocessor.b", artifact_path="preprocessor")
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[0]	validation-rmse:70.04860
[1]	validation-rmse:69.99760
[2]	validation-rmse:69.95275
[3]	validation-rmse:69.91017
[4]	validation-rmse:69.87780
[5]	validation-rmse:69.85012
[6]	validation-rmse:69.82176
[7]	validation-rmse:69.80178
[8]	validation-rmse:69.78116
[9]	validation-rmse:69.76031
[10]	validation-rmse:69.74763
[11]	validation-rmse:69.73369
[12]	validation-rmse:69.71935
[13]	validation-rmse:69.70630
[14]	validation-rmse:69.69626
[15]	validation-rmse:69.68710
[16]	validation-rmse:69.68007
[17]	validation-rmse:69.67383


/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [00:36:14] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[18]	validation-rmse:69.66694
[19]	validation-rmse:69.66157
[20]	validation-rmse:69.65629
[21]	validation-rmse:69.65479
[22]	validation-rmse:69.65368
[23]	validation-rmse:69.65123
[24]	validation-rmse:69.64921
[25]	validation-rmse:69.64640
[26]	validation-rmse:69.64613
[27]	validation-rmse:69.64656
[28]	validation-rmse:69.64444
[29]	validation-rmse:69.64570
[30]	validation-rmse:69.64640
[31]	validation-rmse:69.64216
[32]	validation-rmse:69.64059
[33]	validation-rmse:69.63693
[34]	validation-rmse:69.63811
[35]	validation-rmse:69.63782
[36]	validation-rmse:69.63513
[37]	validation-rmse:69.63615
[38]	validation-rmse:69.63373
[39]	validation-rmse:69.62959
[40]	validation-rmse:69.62692
[41]	validation-rmse:69.62686
[42]	validation-rmse:69.62479
[43]	validation-rmse:69.62387
[44]	validation-rmse:69.62030
[45]	validation-rmse:69.62236
[46]	validation-rmse:69.62079
[47]	validation-rmse:69.61887
[48]	validation-rmse:69.61929
[49]	validation-rmse:69.62207
[50]	validation-rmse:69.62125
[51]	valid

/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [00:36:15] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/25 00:36:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [32]:
logged_model = 'runs:/b1d6141c84984ae783deccf199d5e749/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [33]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: b1d6141c84984ae783deccf199d5e749

In [34]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [35]:
xgboost_model

In [36]:
xgboost_model.predict(valid)

array([39.895706, 28.665522, 31.035696, ..., 29.027748, 24.789118,
       20.100262], shape=(63185,), dtype=float32)

## Deployment

In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [9]:
client.create_experiment(name="my-cool-experiment")

'5'

To get the best run methods you can use:

In [11]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids=1,
    filter_string = "",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
    
)

In [14]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse {run.data.metrics['rmse']:.4f}")

run id: 5e2da22b91d345bfa1deb2d66ecb87e8, rmse 5.4312


## Promote

In [17]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [19]:
run_id = "5e2da22b91d345bfa1deb2d66ecb87e8"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-regressor")

Successfully registered model 'nyc-regressor'.
Created version '1' of model 'nyc-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1748200577919, current_stage='None', description=None, last_updated_timestamp=1748200577919, name='nyc-regressor', run_id='5e2da22b91d345bfa1deb2d66ecb87e8', run_link=None, source='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns/1/5e2da22b91d345bfa1deb2d66ecb87e8/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [1]:
import mlflow

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'file:///home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns'


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

Traceback (most recent call last):
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise 

default artifacts URI: 'file:///home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns/614094180776244277/35fcfc65b87d4f9389e1a6fa5ae53f88/artifacts'


In [9]:
mlflow.search_experiments()

Traceback (most recent call last):
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/pastor/projects/mlops-zoomcamp/.venv/lib/python3.13/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise 

[<Experiment: artifact_location='file:///home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/mlruns/614094180776244277', creation_time=1748208853232, experiment_id='614094180776244277', last_update_time=1748208853232, lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient()

In [16]:
from mlflow.exceptions import MlflowException

try:
    print(client.search_registered_models())
except MlflowException:
    print("It's not possible to access the model registry :(")

[]


## Scenario 2: server local

In [1]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/artifacts_local/0', creation_time=1748225585457, experiment_id='0', last_update_time=1748225585457, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/26 02:19:47 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/05/26 02:19:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/artifacts_local/1/e18eb4175e18465b84b8638f4c1e9b69/artifacts'
🏃 View run youthful-auk-57 at: http://127.0.0.1:5000/#/experiments/1/runs/e18eb4175e18465b84b8638f4c1e9b69
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/artifacts_local/1', creation_time=1748225987526, experiment_id='1', last_update_time=1748225987526, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/artifacts_local/0', creation_time=1748225585457, experiment_id='0', last_update_time=1748225585457, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with model registry

In [6]:
from mlflow.tracking import MlflowClient
client = MlflowClient()

In [7]:
client.search_registered_models()

[]

In [23]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/26 02:27:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748226434774, current_stage='None', description='', last_updated_timestamp=1748226434774, name='iris-classifier', run_id='e18eb4175e18465b84b8638f4c1e9b69', run_link='', source='/home/pastor/projects/mlops-zoomcamp/02_experimental_tracking/artifacts_local/1/e18eb4175e18465b84b8638f4c1e9b69/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>